In [2]:
import pandas as pd
import yfinance as yf
import sys

In [3]:
# Use the following site to get list of all publicly traded Stocks
#http://nxcoreapi.com/symbols.php

#nyse
nyse = pd.read_html('http://nxcoreapi.com/symbols.php?search=&m_exchange=NYSE&m_type=&s_type=exact&m_symbol=on&m_name=on')
#nasdaq
nqnm = pd.read_html('http://nxcoreapi.com/symbols.php?search=&m_exchange=NQNM&m_type=&s_type=exact&m_symbol=on&m_name=on')
#amex
amex = pd.read_html('http://nxcoreapi.com/symbols.php?search=&m_exchange=AMEX&m_type=&s_type=exact&m_symbol=on&m_name=on')

In [4]:
nyse = nyse[0]
nyse.columns = nyse.iloc[0]
nyse = nyse[1:]
nyse

,Exchange,Type,Symbol,Name,Activity,Last Quote,Last Trade
1,NYSE,e,A,"Agilent Technologies, Inc",NaN,2020-08-26,2020-08-25
2,NYSE,e,AA,Alcoa Corp,NaN,2020-08-26,2020-08-25
3,NYSE,e,AAN,"Aaron's, Inc",NaN,2020-08-26,2020-08-25
4,NYSE,e,AAP,"Advance Auto Parts, Inc W/I",NaN,2020-08-26,2020-08-25
5,NYSE,e,AAT,"American Assets Trust, Inc",NaN,2020-08-25,2020-08-25
...,...,...,...,...,...,...,...
3334,NYSE,e,ZTO,"ZTO Express (Cayman), Inc ADS",NaN,2020-08-26,2020-08-26
3335,NYSE,e,ZTR,Virtus Total Return Fund Inc.,NaN,2020-08-26,2020-08-25
3336,NYSE,e,ZTS,"Zoetis, Inc",NaN,2020-08-26,2020-08-25
3337,NYSE,e,ZUO,"Zuora, Inc",NaN,2020-08-26,2020-08-25


In [5]:
nqnm = nqnm[0]
nqnm.columns = nqnm.iloc[0]
nqnm = nqnm[1:]
nqnm

,Exchange,Type,Symbol,Name,Activity,Last Quote,Last Trade
1,NQNM,e,AACG,ATA Creativity Global - American Depositary Sh...,NaN,2020-08-26,2020-08-25
2,NQNM,e,AAL,"American Airlines Group, Inc",NaN,2020-08-26,2020-08-26
3,NQNM,e,AAME,Atlantic American Corp,NaN,2020-08-26,2020-08-25
4,NQNM,e,AAOI,"Applied Optoelectronics, Inc",NaN,2020-08-26,2020-08-25
5,NQNM,e,AAON,"AAON, Inc",NaN,2020-08-26,2020-08-25
...,...,...,...,...,...,...,...
2616,NQNM,e,ZVZZC,NASDAQ TEST STOCK Nextshares Test Security,NaN,NaN,NaN
2617,NQNM,e,ZVZZT,NASDAQ TEST STOCK,NaN,2020-08-26,2020-08-26
2618,NQNM,e,ZXYZ.A,Nasdaq Symbology Test,NaN,NaN,2019-10-29
2619,NQNM,e,ZXZZT,NASDAQ TEST STOCK,NaN,NaN,2020-08-25


In [6]:
amex = amex[0]
amex.columns = amex.iloc[0]
amex = amex[1:]
amex

,Exchange,Type,Symbol,Name,Activity,Last Quote,Last Trade
1,AMEX,e,AAMC,Altisource Asset Mgmt Corp,NaN,2020-08-26,2020-08-25
2,AMEX,e,AAU,"Almaden Minerals, Ltd",NaN,2020-08-26,2020-08-25
3,AMEX,e,ACU,Acme United Corp,NaN,2020-08-26,2020-08-25
4,AMEX,e,ACY,AeroCentury Corp,NaN,2020-08-26,2020-08-25
5,AMEX,e,AE,"Adams Resources & Energy, Inc",NaN,2020-08-25,2020-08-25
...,...,...,...,...,...,...,...
316,AMEX,e,YCBD,"cbdMD, Inc.",NaN,2020-08-26,2020-08-26
317,AMEX,e,YCBD.PR.A,"cbdMD, Inc. 8.0% Series A Cumulative Convertib...",NaN,2020-08-25,2020-08-25
318,AMEX,e,YUMA,"Yuma Energy, Inc",NaN,NaN,NaN
319,AMEX,e,ZDGE,"Zedge, Inc",NaN,2020-08-26,2020-08-25


In [7]:
stocks = nyse
stocks = stocks.append(nqnm)
stocks = stocks.append(amex)
# stocks.tail()
stocks


,Exchange,Type,Symbol,Name,Activity,Last Quote,Last Trade
1,NYSE,e,A,"Agilent Technologies, Inc",NaN,2020-08-26,2020-08-25
2,NYSE,e,AA,Alcoa Corp,NaN,2020-08-26,2020-08-25
3,NYSE,e,AAN,"Aaron's, Inc",NaN,2020-08-26,2020-08-25
4,NYSE,e,AAP,"Advance Auto Parts, Inc W/I",NaN,2020-08-26,2020-08-25
5,NYSE,e,AAT,"American Assets Trust, Inc",NaN,2020-08-25,2020-08-25
...,...,...,...,...,...,...,...
316,AMEX,e,YCBD,"cbdMD, Inc.",NaN,2020-08-26,2020-08-26
317,AMEX,e,YCBD.PR.A,"cbdMD, Inc. 8.0% Series A Cumulative Convertib...",NaN,2020-08-25,2020-08-25
318,AMEX,e,YUMA,"Yuma Energy, Inc",NaN,NaN,NaN
319,AMEX,e,ZDGE,"Zedge, Inc",NaN,2020-08-26,2020-08-25


In [8]:
stocks = stocks[['Symbol', 'Name', 'Exchange']]
stocks.reset_index(inplace=True)
stocks = stocks.drop(columns=['index'])
stocks

,Symbol,Name,Exchange
0,A,"Agilent Technologies, Inc",NYSE
1,AA,Alcoa Corp,NYSE
2,AAN,"Aaron's, Inc",NYSE
3,AAP,"Advance Auto Parts, Inc W/I",NYSE
4,AAT,"American Assets Trust, Inc",NYSE
...,...,...,...
6273,YCBD,"cbdMD, Inc.",AMEX
6274,YCBD.PR.A,"cbdMD, Inc. 8.0% Series A Cumulative Convertib...",AMEX
6275,YUMA,"Yuma Energy, Inc",AMEX
6276,ZDGE,"Zedge, Inc",AMEX


In [14]:
symbol = yf.Ticker('NTUS')
symbol.info

{'zip': '94566',
 'sector': 'Healthcare',
 'fullTimeEmployees': 1484,
 'longBusinessSummary': "Natus Medical Incorporated provides medical device solutions focusing on the diagnosis and treatment of patients with central nervous and sensory system disorders worldwide. It offers products and services used for the screening, detection, treatment, monitoring, and tracking of common medical ailments in newborn care, hearing impairment, neurological and neurosurgical treatments, epilepsy, sleep disorders, and neuromuscular diseases. The company also provides computerized neurodiagnostic systems for audiology, neurology, polysomnography, and neonatology; and software systems for managing and tracking disorders and diseases for public health laboratories. In addition, it offers electroencephalography, long term monitoring, intensive care unit monitoring, electromyography, sleep analysis or polysomnography, and intra-operative monitoring solutions. Further, the company provides hearing screeni

In [9]:
# msft.info

In [10]:
# type(msft.history(period="max"))
# msft.history(period='max')

In [11]:
# data = yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30")
# data['Close']

In [15]:
i = 0
for row in stocks.itertuples():
    print (i)
    symbol = row[1]
    symbol = yf.Ticker(symbol)
    try:
        stocks.loc[i, 'industry'] = symbol.info['industry']
    except:
        stocks.loc[i, 'industry'] = 'NA'
        e = sys.exc_info()[0]
        print(e)
   
    try:
        stocks.loc[i, 'sector'] = symbol.info['sector']
    except:
        stocks.loc[i, 'sector'] = 'NA'
        e = sys.exc_info()[0]
        print(e)

    try:
        stocks.loc[i, 'dividend_rate'] = symbol.info['dividendRate']
    except:
        stocks.loc[i, 'dividend_rate'] = 'NA'
        e = sys.exc_info()[0]
        print(e)

    try:
        stocks.loc[i, 'dividend_date'] = symbol.info['exDividendDate']
    except:
        stocks.loc[i, 'dividend_date'] = 'NA'
        e = sys.exc_info()[0]
        print(e)

    i += 1
    

0
<class 'urllib.error.HTTPError'>
1
2
3
4
5
6
7
8
9
10
11
12
13
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
14
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
15
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
16
17
18
19
20
21
22
23
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
24
25
26
27
28
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
29
30
31
32
33
34
35
36
37
38
39
40
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
41
42
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
43
<class 'urllib.error.HTTPError'>
44
45
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
46
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
47
48
49
50
<class 'ValueError'>
<class 'ValueE

<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
246
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
247
248
<class 'KeyError'>
249
250
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'urllib.error.HTTPError'>
251
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
252
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
253
254
255
256
257
258
259
260
261
<class 'IndexError'>
<class 'KeyError'>
<class 'IndexError'>
262
263
264
265
266
<class 'KeyError'>
<class 'KeyError'>
267
<class 'KeyError'>
<class 'KeyError'>
268
269
270
271
272
273
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
274
<class 'KeyError'>
<class 'KeyError'>
275
276
277
278
279
280
281
282
283
<class 'KeyError'>
<class 'KeyError'>
284
285
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
286
287


<class 'ValueError'>
546
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
547
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
548
<class 'requests.exceptions.ChunkedEncodingError'>
549
550
551
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
552
553
554
555
<class 'urllib.error.HTTPError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
556
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
557
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
558
559
560
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
561
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
562
563
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
564
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
565
566
567
568


<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
776
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
777
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
778
<class 'urllib.error.URLError'>
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
779
780
781
782
783
784
785
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
786
787
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
788
789
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
790
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
791
792
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
793
794
795
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
796
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
797
<class 'ValueEr

<class 'IndexError'>
<class 'IndexError'>
1015
1016
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
1017
1018
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
<class 'KeyError'>
1019
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
1020
1021
1022
1023
1024
1025
1026
1027
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
1028
1029
1030
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
1031
1032
1033
1034
<class 'KeyError'>
<class 'IndexError'>
<class 'IndexError'>
<class 'KeyError'>
1035
1036
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
<class 'IndexError'>
1037
1038
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
<class 'ValueError'>
1049
1050
1051
<class 'IndexError'>
<class '

...

<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6001
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6002
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6003
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6004
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6005
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6006
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6007
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class '

<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6061
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6062
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6063
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6064
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6065
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6066
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6067
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class '

<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6121
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6122
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6123
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6124
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6125
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6126
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6127
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class '

<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6181
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6182
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6183
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6184
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6185
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6186
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6187
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class '

<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6241
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6242
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6243
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6244
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6245
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6246
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
6247
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class 'urllib.error.HTTPError'>
<class '

In [17]:
stocks.to_csv('info.csv')